<a href="https://colab.research.google.com/github/TurkuNLP/textual-data-analysis-course/blob/main/tda_2023_exercises_1_solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YLE news data

* We'll be using data from YLE news for many of the demos
* This data is gathered from 2021 YLE RSS feed
* It's here: http://dl.turkunlp.org/TKO_8964_2023/

# Task 1

* Grab the data with `wget` and give it a look
* Think what kinds of NLP tasks you could use the data for?

In [1]:
!wget http://dl.turkunlp.org/TKO_8964_2023/news-fi-2021.jsonl

--2023-01-15 12:30:20--  http://dl.turkunlp.org/TKO_8964_2023/news-fi-2021.jsonl
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36139303 (34M) [application/octet-stream]
Saving to: ‘news-fi-2021.jsonl’

news-fi-2021.jsonl  100%[===================>]  34.46M  32.2MB/s    in 1.1s    

2023-01-15 12:30:21 (32.2 MB/s) - ‘news-fi-2021.jsonl’ saved [36139303/36139303]



The data can be used for, e.g.:


*   Retrieval (as we did)
*   Classification / topic labeling (there seem to be topic labels)
*   Summarization



# Task 2

* Load the data as a HuggingFace dataset
* Remember to pip-install datasets first
* https://huggingface.co/docs/datasets/loading#json

In [2]:
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
import datasets
dset=datasets.load_dataset("json", data_files="news-fi-2021.jsonl")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ebbaba727a3fbc92/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
for item in dset["train"]:
    print(item.keys())
    break

dict_keys(['summary', 'tags', 'text', 'timestamp', 'title', 'url'])


# Task 3

- Try some keyword search on the data
- Run the data through the trusty CountVectorizer
- Do some keyword search, e.g. look for "Turku" AND "silta" and print the titles of the news that match (do the search on the text field, though)
- Some hints:
  - if you just copy the code from the lecture, you will get out-of-memory error
  - can you think why? what is the data structure returned by the vectorizer?
  - the vocabulary of the vectorizer is in `cv.vocabulary_`
  - you may still need `.todense()` but maybe in a different spot
  - you may need `.nonzero()` to gather the matching documents
  


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(lowercase=False,binary=True)
td_matrix=cv.fit_transform(item["text"] for item in dset["train"]).T #here one should remove todense() not to blow memory limit

In [6]:
u=td_matrix[cv.vocabulary_["Turku"]]
m=td_matrix[cv.vocabulary_["silta"]]

In [7]:
result=u.todense() & m.todense() # boolean search does not work on sparse representation, so one needs to make the rows dense one at a time, this is comparatively cheap
result.nonzero()

(array([0, 0, 0]), array([1651, 1660, 3997]))

In [8]:
for j in result.nonzero()[1]:
    print(dset["train"][int(j)]["title"])

Tunteita herättänyt ja ilkivallan kohteeksikin joutunut Turun Teatterisillan sateenkaaritähti poistetaan perjantaina
Yli 21 miljoonaa maksanut Logomon silta hämmentää Turussa: Miten sinne pääsee, miksi silta loppuu kesken? Saimme vastauksia kysymyksiin
Vuosien työ tulee päätökseen – Turun Logomon silta sai viimein avajaispäivän


# Task 4

* This is for those who run a little ahead
* Run the dataset through a HuggingFace pipeline as follows:
  * Model: `xlm-roberta-base` (or any other similar model of your choice)
  * Task: `feature-extraction`
  * Relevant documentation: https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/pipelines#pipeline-batching
  * You will likely want to use the GPU
  * Make sure you understand what the return values are


In [9]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.8 MB/s eta 0:00:00


In [10]:
import transformers


In [17]:
p=transformers.pipeline(task="feature-extraction",model="xlm-roberta-base",return_tensors=True, device=0)
for x in p(transformers.pipelines.pt_utils.KeyDataset(dset["train"], "text"), batch_size=8, truncation="only_first"):
    print(x.shape)
    break


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 512, 768])


What we get back is an iterable with embedding for each item. Here the embedding seems to be 512x768 where 512 is typically the maximum sequence length, and 768 the embedding dimensionality; so this is an embedding of each token in the first 512 tokens in the text

In [18]:
print(x.device)

cpu
